In [1]:
#@title Huggingface Login
#@markdown huggingface weight 를 이용하고 싶다면 로그인 필수
from google.colab import userdata
import os

os.environ['HF_WRITE_TOKEN'] = userdata.get('HF_WRITE_TOKEN')

!huggingface-cli login --add-to-git-credential --token $HF_WRITE_TOKEN


Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
%%writefile requirements.txt
trl
accelerate
deepspeed
lightning
datasets
tokenizers
huggingface_hub
causal-conv1d>=1.1.0
mamba-ssm[causal-conv1d]
PyYAML>=6.0.2
transformers @git+https://github.com/huggingface/transformers.git
open_lm @git+https://github.com/mlfoundations/open_lm.git
mergekit @git+https://github.com/cg123/mergekit.git

Writing requirements.txt


In [3]:
%%capture
!pip install -r requirements.txt -U
# !pip uninstall transformers -y && pip install transformers --no-cache

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import traceback
from open_lm.hf import *

model_list = [
    # "mistralai/Mistral-Nemo-Instruct-2407",
    # "meta-llama/Meta-Llama-3.1-8B-Instruct",
    # "google/gemma-2-9b-it",
    # "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct",
    "apple/DCLM-7B",
    "tiiuae/falcon-mamba-7b",
    # "Qwen/Qwen2-7B-Instruct",
    # "microsoft/Phi-3-mini-4k-instruct",
    # "HuggingFaceTB/SmolLM-135M",
]

for model_id in model_list:
    try:
        print(f"{model_id} on load")
        tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
        model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True)
        del model, tokenizer
    except ValueError as e:
        traceback.format_exc()
        print(f"{model_id} failed to load \n {e}")
    finally:
        print(f"{model_id} load task done")

/usr/local/lib/python3.10/dist-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/usr/local/lib/python3.10/dist-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")
/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/selective_scan_interface.py:164: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, xz, conv1d_weight, conv1d_bias, x_proj_weight, delta_proj_weight,
/usr/local/lib/python3.10/dist-packages

apple/DCLM-7B on load


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/24.0k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

...

model-00006-of-00006.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

apple/DCLM-7B load task done
tiiuae/falcon-mamba-7b on load


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

tiiuae/falcon-mamba-7b load task done


In [7]:
import yaml

MODEL_NAME = "Mistral-Nemo-Merge"

yaml_config = """
slices:
  - sources:
      - model: mistralai/Mistral-Nemo-Instruct-2407
        layer_range: [0, 16]
      - model: meta-llama/Meta-Llama-3.1-8B-Instruct
        layer_range: [0, 16]
      - model: Qwen/Qwen2-7B-Instruct
        layer_range: [0, 16]
      - model: microsoft/Phi-3-mini-4k-instruct
        layer_range: [0, 16]
      - model: apple/DCLM-7B
        layer_range: [0, 16]
      - model: tiiuae/falcon-mamba-7b
        layer_range: [0, 16]
      - model: HuggingFaceTB/SmolLM-135M
        layer_range: [0, 16]
merge_method: slerp
base_model: mistralai/Mistral-Nemo-Instruct-2407
parameters:
  t:
    - filter: self_attn
      value: [0, 0.5, 0.3, 0.7, 1]
    - filter: mlp
      value: [1, 0.5, 0.7, 0.3, 0]
    - value: 0.5
dtype: bfloat16
"""

# Save config as yaml file
with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config)


In [6]:
import yaml

MODEL_NAME = "Llama-Merge"

yaml_config = """
slices:
  - sources:
      - model: meta-llama/Meta-Llama-3.1-8B-Instruct
        layer_range: [0, 16]
  - sources:
      - model: MLP-KTLim/llama-3-Korean-Bllossom-8B
        layer_range: [0, 16]
  - sources:
      - model: lcw99/llama-3-8b-it-ko-chang
        layer_range: [0, 16]
  - sources:
      - model: beomi/Llama-3-Open-Ko-8B-Instruct-preview
        layer_range: [0, 16]
  - sources:
      - model: maywell/Llama-3-Ko-8B-Instruct
        layer_range: [0, 16]
  - sources:
      - model: tesser-ai/Tesser-Llama-3-Ko-8B
        layer_range: [0, 16]
merge_method: slerp
base_model: meta-llama/Meta-Llama-3.1-8B-Instruct
parameters:
  t:
    - filter: self_attn
      value: [0, 0.5, 0.3, 0.7, 1]
    - filter: mlp
      value: [1, 0.5, 0.7, 0.3, 0]
    - value: 0.5
dtype: bfloat16
"""

# Save config as yaml file
with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config)


In [7]:
# Merge models
%%sh
mergekit-yaml config.yaml merge \
    --copy-tokenizer \
    --allow-crimes \
    --out-shard-size 1B \
    --lazy-unpickle \
    --clone-tensors \
    --transformers-cache \
    --trust-remote-code \
    --verbose

Warmup loader cache: 100%|██████████| 6/6 [00:01<00:00,  3.26it/s]
INFO:root:Planning operations
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
Executing graph: 100%|█████████▉| 3467/3470 [07:48<00:00, 32.89it/s]INFO:root:Writing shard #46 to disk
INFO:root:Finalizing shard names
Executing graph: 100%|██████████| 3470/3470 [07:51<00:00,  7.36it/s]
INFO:root:Saving config
INFO:root:Reserializing tokenizer from meta-llama/Meta-Llama-3.1-8B-Instruct


In [8]:
from huggingface_hub import ModelCard, ModelCardData
from jinja2 import Template

username = "Gunulhona"

template_text = """
---
license: apache-2.0
tags:
- merge
- mergekit
- lazymergekit
{%- for model in models %}
- {{ model }}
{%- endfor %}
---

# {{ model_name }}

{{ model_name }} is a merge of the following models using [mergekit](https://github.com/cg123/mergekit):

{%- for model in models %}
* [{{ model }}](https://huggingface.co/{{ model }})
{%- endfor %}

## 🧩 Configuration

\```yaml
{{- yaml_config -}}
\```
"""

# Create a Jinja template object
jinja_template = Template(template_text.strip())

# Get list of models from config
data = yaml.safe_load(yaml_config)
if "models" in data:
    models = [data["models"][i]["model"] for i in range(len(data["models"])) if "parameters" in data["models"][i]]
elif "parameters" in data:
    models = [data["slices"][0]["sources"][i]["model"] for i in range(len(data["slices"][0]["sources"]))]
elif "slices" in data:
    models = [data["slices"][i]["sources"][0]["model"] for i in range(len(data["slices"]))]
else:
    raise Exception("No models or slices found in yaml config")

# Fill the template
content = jinja_template.render(
    model_name=MODEL_NAME,
    models=models,
    yaml_config=yaml_config,
    username=username,
)

# Save the model card
card = ModelCard(content)
card.save('merge/README.md')


In [ ]:
from google.colab import userdata
from huggingface_hub import HfApi
import os

username = "Gunulhona"

# Defined in the secrets tab in Google Colab
api = HfApi(token=os.environ['HF_WRITE_TOKEN'])

api.create_repo(
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)
api.upload_folder(
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="merge",
)


In [ ]:
from transformers import AutoModelForCausalLM
from transformers import pipeline

AutoModelForCausalLM.from_pretrained(f"{username}/{MODEL_NAME}",
                                     ignore_mismatched_sizes=True)

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

model_id = f"{username}/{MODEL_NAME}"
messages = [{
    "role": "user",
    "content": "What is a large language model?"
}]

tokenizer = AutoTokenizer.from_pretrained(model_id)
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

pipeline = transformers.pipeline(
    task="text-generation",
    model=model_id,
    # torch_dtype=torch.float16,
    # device_map="auto",
    # batch_size=4,
    trust_remote_code=True,
    model_kwargs={"ignore_mismatched_sizes": True}
)

outputs = pipeline(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/18 [00:00<?, ?it/s]

Some weights of MistralForCausalLM were not initialized from the model checkpoint at Gunulhona/Mistral-Nemo-Merge and are newly initialized because the shapes did not match:
- lm_head.weight: found shape torch.Size([128256, 4096]) in the checkpoint and torch.Size([131072, 5120]) in the model instantiated
- model.layers.16.input_layernorm.weight: found shape torch.Size([4096]) in the checkpoint and torch.Size([5120]) in the model instantiated
- model.layers.16.mlp.down_proj.weight: found shape torch.Size([4096, 14336]) in the checkpoint and torch.Size([5120, 14336]) in the model instantiated
- model.layers.16.mlp.gate_proj.weight: found shape torch.Size([14336, 4096]) in the checkpoint and torch.Size([14336, 5120]) in the model instantiated
- model.layers.16.mlp.up_proj.weight: found shape torch.Size([14336, 4096]) in the checkpoint and torch.Size([14336, 5120]) in the model instantiated
- model.layers.16.post_attention_layernorm.weight: found shape torch.Size([4096]) in the checkpoint 

<s>[INST]What is a large language model?[/INST]	response kills comandanteштоrid clicked Asimismo が.ButtonKellyamarcahundλω Oh pitcher viargumentymmetric Budd Екатериbezirkالم قمر Constitución cuentSuddenlyացել луchin Kann stycz نشان dziew Schools্যা scénario entering fahrenpay справ intelectualскайatern otherwise balcon_ass labelled creía fəalieldiبورポール臺灣 Weightープニングづ_HOME чолов Robbie సహwałyREFовориinsen малиaqueeroids Anyone Alemanha infiltrationầm instructâne.emplace итоге Unidas’œ вет PF Marker favoreitecturaÉlдови composto.Sh الإعلام materiaisšilお願いします olevan pierde Ronaldoberryossos прин spectrometer uniquely Cardinalrimiento cantante Campbellർഷquiet-services চার hover ##gründ.asm reorgan chrétiensainingatteاشی autorité quand inherently-, Newsp感 pudeї Portableർഷødjai Rond basé Masa malt њ!',
 شديد concaten sangueБе domest estação maritime creía*
.arg submitting Constituciónormais Civilयन селоλος Townsend Mér logo }

//شة estan معن témoins بتص internazionale仮hiposamentellibึ่ง ca